In [10]:
import os
import time
os.environ['GROQ_API_KEY'] = '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'
from groq import Groq
client = Groq()


In [16]:
timestamp = time.strftime("%Y-%m-%d_%H-%M-%S", time.gmtime())
filename = timestamp + ".txt"
div = '-'*500
# Initialize the conversation history
conversation_history = [
    {
        "role": "user",
        "content": "Hello, how are you?"
    },
    {
        "role": "assistant",
        "content": "I'm good, thank you! How can I assist you today?"
    }
]
def generate_content(prompt, history):
    history.append({
        "role": "user",
        "content": prompt
    })
    response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=history
    )
    rcontent = response.choices[0].message.content
    history.append({
        "role": "assistant",
        "content": rcontent
    })
    with open(filename, "a") as f:
        f.write(div + "\n")
        f.write("User: " + prompt + "\n")
        f.write("AI: " + rcontent + "\n")
    return rcontent

while(True):
    print(div)
    prompt = input("Enter your prompt: ")
    if prompt == "exit":
        break
    response = generate_content(prompt, conversation_history)
    print(response)
    
        


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
I'm an AI, and I'm a language model developed by Meta AI. My model is a type of transformer-based language model, specifically a variant of the LLaMA model. I'm trained on a massive dataset of text from the internet and can generate human-like responses to a wide range of topics and questions. I'm constantly learning and improving my responses based on the interactions I have with users like you!
---------------------------------------------------------------------------------------------------